In [4]:
import numpy as np
import pandas as pd
import sqlite3

In [ ]:
conn = sqlite3.connect('data/coffee_data.db')
query = "SELECT * FROM coffee_samples"

dbf = pd.read_sql_query(query, conn)
conn.close()

print(dbf.head())

   id country_of_origin number_of_bags                 farm_name bag_weight  \
0   1          Thailand              3         Pichet Klaphithak      20 kg   
1   2          Thailand              2             Ching Saethao      25 kg   
2   3            Brazil              3         Fazenda Guariroba      10 kg   
3   4          Thailand              1  Coffee De Hmong Bio Farm      10 kg   
4   5          Thailand              2     Wichai Kamnerdmongkon      25 kg   

  lot_number                      in_country_partner  \
0        TCE                Coffee Quality Institute   
1        TCE                Coffee Quality Institute   
2    RR24009                    Coffee Quality Union   
3       2024  Specialty Coffee Institute of Thailand   
4        TCE                Coffee Quality Institute   

                                              mill harvest_year ico_number  \
0  Big Black Box's Dry Mill Plant Sai Noi District  2023 / 2024              
1                     Big Black 

In [ ]:
df = pd.read_csv('data/arabica_data_cleaned.csv')
print(df.head())

   Unnamed: 0  Species                     Owner Country.of.Origin  \
0           1  Arabica                 metad plc          Ethiopia   
1           2  Arabica                 metad plc          Ethiopia   
2           3  Arabica  grounds for health admin         Guatemala   
3           4  Arabica       yidnekachew dabessa          Ethiopia   
4           5  Arabica                 metad plc          Ethiopia   

                                  Farm.Name Lot.Number       Mill ICO.Number  \
0                                 metad plc        NaN  metad plc  2014/2015   
1                                 metad plc        NaN  metad plc  2014/2015   
2  san marcos barrancas "san cristobal cuch        NaN        NaN        NaN   
3     yidnekachew dabessa coffee plantation        NaN    wolensu        NaN   
4                                 metad plc        NaN  metad plc  2014/2015   

                                 Company       Altitude  ...  Color  \
0      metad agricultural d

In [13]:
print(df.info())
print(dbf.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1311 entries, 0 to 1310
Data columns (total 44 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Unnamed: 0             1311 non-null   int64  
 1   Species                1311 non-null   object 
 2   Owner                  1304 non-null   object 
 3   Country.of.Origin      1310 non-null   object 
 4   Farm.Name              955 non-null    object 
 5   Lot.Number             270 non-null    object 
 6   Mill                   1001 non-null   object 
 7   ICO.Number             1163 non-null   object 
 8   Company                1102 non-null   object 
 9   Altitude               1088 non-null   object 
 10  Region                 1254 non-null   object 
 11  Producer               1081 non-null   object 
 12  Number.of.Bags         1311 non-null   int64  
 13  Bag.Weight             1311 non-null   object 
 14  In.Country.Partner     1311 non-null   object 
 15  Harv

In [ ]:
def split_altitude(altitude):
    if pd.isna(altitude) or altitude == '' or altitude == 'n/a':
        return np.nan, np.nan, np.nan
    altitude = altitude.replace(',', '')
    altitude = altitude.replace('above', '')
    altitude = altitude.replace('Above', '')
    altitude = altitude.replace('>', '')
    altitude = altitude.replace(' a ', '-')    
    altitude = altitude.replace(' A ', '-')    
    altitude = altitude.replace('~', '-')
    if '-' in altitude:
        min_altitude, max_altitude = altitude.split('-')
        return float(min_altitude), float(max_altitude), (float(min_altitude) + float(max_altitude)) / 2
    else:
        return float(altitude), float(altitude), float(altitude)
    


In [32]:
dbf[['altitude_low_meters', 'altitude_high_meters', 'altitude_mean_meters']] = dbf['altitude'].apply(lambda x: pd.Series(split_altitude(x)))
print(dbf[['altitude_low_meters', 'altitude_high_meters', 'altitude_mean_meters']].head())

   altitude_low_meters  altitude_high_meters  altitude_mean_meters
0               1403.0                1403.0                1403.0
1               1350.0                1350.0                1350.0
2               1100.0                1100.0                1100.0
3               1443.0                1443.0                1443.0
4               1445.0                1445.0                1445.0


     altitude_low_meters  altitude_high_meters  altitude_mean_meters
0                 1403.0                1403.0                1403.0
1                 1350.0                1350.0                1350.0
2                 1100.0                1100.0                1100.0
3                 1443.0                1443.0                1443.0
4                 1445.0                1445.0                1445.0
..                   ...                   ...                   ...
381                250.0                 250.0                 250.0
382                220.0                 220.0                 220.0
383               1200.0                1200.0                1200.0
384                250.0                 380.0                 315.0
385               1000.0                1200.0                1100.0

[386 rows x 3 columns]


In [44]:
columns_to_convert = [
    "number_of_bags", 'aroma', 'uniformity', 'flavor', 'clean_cup',
    'aftertaste', 'sweetness', 'acidity', 'body', 'defects',
    'balance', 'total_cup_points', 'moisture'
]

dbf[columns_to_convert] = dbf[columns_to_convert].replace(['', 'n/a', 'NA', 'null'], '0')

for col in columns_to_convert:
    dbf[col] = pd.to_numeric(dbf[col], errors='coerce')


In [45]:
print(dbf.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 386 entries, 0 to 385
Data columns (total 39 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   id                    386 non-null    int64  
 1   country_of_origin     386 non-null    object 
 2   number_of_bags        384 non-null    float64
 3   farm_name             386 non-null    object 
 4   bag_weight            386 non-null    object 
 5   lot_number            386 non-null    object 
 6   in_country_partner    386 non-null    object 
 7   mill                  386 non-null    object 
 8   harvest_year          386 non-null    object 
 9   ico_number            386 non-null    object 
 10  grading_date          386 non-null    object 
 11  company               386 non-null    object 
 12  owner                 386 non-null    object 
 13  altitude              386 non-null    object 
 14  variety               386 non-null    object 
 15  region                3

In [62]:
dbf['category_one_defects'] = dbf['category_one_defects'].str.replace(' full defects', '', regex=False)
dbf['category_two_defects'] = dbf['category_two_defects'].str.replace(' full defects', '', regex=False)


In [78]:
dbf[['category_one_defects', 'category_two_defects']].replace(['', 'n/a', 'NA', 'null'], '0')
dbf['category_one_defects'] = pd.to_numeric(dbf['category_one_defects'], errors='coerce')
dbf['category_two_defects'] = pd.to_numeric(dbf['category_two_defects'], errors='coerce')

In [84]:
dbf['category_one_defects'] = dbf['category_one_defects'].fillna(0)
dbf['category_two_defects'] = dbf['category_two_defects'].fillna(0)

In [85]:
dbf['category_one_defects'] = dbf['category_one_defects'].astype('int')
dbf['category_two_defects'] = dbf['category_two_defects'].astype('int')

In [86]:
print(dbf.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 386 entries, 0 to 385
Data columns (total 39 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   id                    386 non-null    int64  
 1   country_of_origin     386 non-null    object 
 2   number_of_bags        384 non-null    float64
 3   farm_name             386 non-null    object 
 4   bag_weight            386 non-null    object 
 5   lot_number            386 non-null    object 
 6   in_country_partner    386 non-null    object 
 7   mill                  386 non-null    object 
 8   harvest_year          386 non-null    object 
 9   ico_number            386 non-null    object 
 10  grading_date          386 non-null    object 
 11  company               386 non-null    object 
 12  owner                 386 non-null    object 
 13  altitude              386 non-null    object 
 14  variety               386 non-null    object 
 15  region                3

In [87]:
dbf = dbf.rename(columns={
    'country_of_origin': 'Country.of.Origin',
    'number_of_bags': 'Number.of.Bags',
    'farm_name': 'Farm.Name',
    'bag_weight': 'Bag.Weight',
    'lot_number': 'Lot.Number',
    'in_country_partner': 'In.Country.Partner',
    'mill': 'Mill',
    'harvest_year': 'Harvest.Year',
    'ico_number': 'ICO.Number',
    'grading_date': 'Grading.Date',
    'company': 'Company',
    'owner': 'Owner',
    'altitude': 'Altitude',
    'variety': 'Variety',
    'region': 'Region',
    'producer': 'Producer',
    'processing_method': 'Processing.Method',
    'aroma': 'Aroma',
    'uniformity': 'Uniformity',
    'flavor': 'Flavor',
    'clean_cup': 'Clean.Cup',
    'aftertaste': 'Aftertaste',
    'sweetness': 'Sweetness',
    'acidity': 'Acidity',
    'body': 'Body',
    'defects': 'Defects',
    'balance': 'Balance',
    'total_cup_points': 'Total.Cup.Points',
    'moisture': 'Moisture',
    'color': 'Color',
    'category_one_defects': 'Category.One.Defects',
    'category_two_defects': 'Category.Two.Defects',
    'quakers': 'Quakers',
    'altitude_low_meters': 'altitude_low_meters',
    'altitude_high_meters': 'altitude_high_meters',
    'altitude_mean_meters': 'altitude_mean_meters'
})


In [88]:
dbf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 386 entries, 0 to 385
Data columns (total 39 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   id                    386 non-null    int64  
 1   Country.of.Origin     386 non-null    object 
 2   Number.of.Bags        384 non-null    float64
 3   Farm.Name             386 non-null    object 
 4   Bag.Weight            386 non-null    object 
 5   Lot.Number            386 non-null    object 
 6   In.Country.Partner    386 non-null    object 
 7   Mill                  386 non-null    object 
 8   Harvest.Year          386 non-null    object 
 9   ICO.Number            386 non-null    object 
 10  Grading.Date          386 non-null    object 
 11  Company               386 non-null    object 
 12  Owner                 386 non-null    object 
 13  Altitude              386 non-null    object 
 14  Variety               386 non-null    object 
 15  Region                3

In [89]:
dbf = dbf.drop(columns=['id', 'status', 'overall'])


In [91]:
for col in df.columns:
    if col not in dbf.columns:
        dbf[col] = np.nan

dbf = dbf[df.columns]

In [92]:
final_df = pd.concat([df, dbf], ignore_index=True)

In [93]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1697 entries, 0 to 1696
Data columns (total 44 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Unnamed: 0             1311 non-null   float64
 1   Species                1311 non-null   object 
 2   Owner                  1690 non-null   object 
 3   Country.of.Origin      1696 non-null   object 
 4   Farm.Name              1341 non-null   object 
 5   Lot.Number             656 non-null    object 
 6   Mill                   1387 non-null   object 
 7   ICO.Number             1549 non-null   object 
 8   Company                1488 non-null   object 
 9   Altitude               1474 non-null   object 
 10  Region                 1640 non-null   object 
 11  Producer               1467 non-null   object 
 12  Number.of.Bags         1695 non-null   float64
 13  Bag.Weight             1697 non-null   object 
 14  In.Country.Partner     1697 non-null   object 
 15  Harv

In [94]:
final_df.to_csv('/Users/admin/Documents/PICT/Year 1 Sem 1/ICT/Data Mining and Warehousing/Project/Brewlytics/data/combined_data.csv', index=False)